In [1]:
#Importing Neccessary Libraries such as Numpy,Pandas,Matplotlib,seaborn

import numpy as np   
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Ignoring warnings 

import warnings   
warnings.filterwarnings("ignore")

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [4]:
app=pd.read_csv(r"D:\Downloads1\Bank Dataset\application_data.csv")
prev_app=pd.read_csv(r"D:\Downloads1\Bank Dataset\previous_application.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Downloads1\\Bank Dataset\\application_data.csv'

In [ ]:
app.head()

#Feature Selection

In [ ]:
app.columns

In [ ]:
app.shape

In [ ]:
msng_info=pd.DataFrame(app.isnull().sum().sort_values()).reset_index() #creating a dataframe for missing values
msng_info.rename(columns={"index":"col_name",0:'null_count'},inplace=True) #rename column name
msng_info['msng_pct']=msng_info['null_count']/app.shape[0]*100 #create a new column for %

In [ ]:
msng_info.head()

In [ ]:
msng_info.to_excel(r"D:\Downloads1\Bank Dataset\missing_info.xlsx",index=False) #creating a excel file

In [ ]:
msng_col=msng_info[msng_info['msng_pct']>=40]['col_name'].to_list() #create a new name & put all the msing value rows 
#above 40% and convert into list
len(msng_col)

In [ ]:
app_msng_remove=app.drop(labels=msng_col,axis=1)
app_msng_remove.shape

In [ ]:
app_msng_remove.head()

In [ ]:
flag_col = []          #creating a df of flag colmns and storing all the Flagcolumns init 
for col in app_msng_remove.columns:
    if col.startswith("FLAG_"):
        flag_col.append(col)

In [ ]:
len(flag_col)


In [ ]:
app_msng_remove.head()

In [ ]:
flag_tgt_col = app_msng_remove[flag_col+['TARGET']] # creating a new df and adding target colmn in the end to plot graph
flag_tgt_col.head()

In [ ]:
sns.countplot(data=flag_tgt_col,x='FLAG_CONT_MOBILE',hue='TARGET')

In [ ]:
plt.figure(figsize=(20,25))

for i,col in enumerate(flag_col):
    plt.subplot(7,4,i+1)
    sns.countplot(data=flag_tgt_col,x=col,hue='TARGET')

In [ ]:
flag_tgt_col.columns

In [ ]:
flag_corr=['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE',
       'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL','TARGET']

flag_corr_df = app_msng_remove[flag_corr]
round(flag_corr_df.corr(),2)# correlation between cols by 2 decimal places
# the corr greater & less than 0.8 are given importance

In [ ]:
flag_corr_df.head()

In [ ]:
corr_df=round(flag_corr_df.corr(),2) #creating df for heatmap

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(corr_df,cmap='coolwarm',linewidth=.5,annot=True)

In [ ]:
# flag own car & flag rEALITY IS MISSING AS THE values are in yes or no not 0 & 1
flag_corr_df.groupby(['FLAG_OWN_CAR']).size()

In [ ]:
flag_corr_df["FLAG_OWN_CAR"]=flag_corr_df["FLAG_OWN_CAR"].replace ({'N':0,'Y':1})
flag_corr_df["FLAG_OWN_REALTY"]=flag_corr_df["FLAG_OWN_REALTY"].replace ({'N':0,'Y':1})


In [ ]:
flag_corr_df.groupby(['FLAG_OWN_CAR']).size()

In [ ]:
corr_df=round(flag_corr_df.corr(),2) #creating df for heatmap

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(corr_df,cmap='coolwarm',linewidth=.5,annot=True)

In [ ]:
#As theese columns have no corelation so we drop all columns except Target

In [ ]:
app_flag_remove=app_msng_remove.drop(labels=flag_col,axis=1)
app_flag_remove.shape

In [ ]:
app_flag_remove.head()

In [ ]:
sns.heatmap(round(app_flag_remove[['EXT_SOURCE_2','EXT_SOURCE_3','TARGET']].corr(),2),cmap='coolwarm',linewidth=.5,annot=True)

In [ ]:
app_score_col_remove=app_flag_remove.drop(['EXT_SOURCE_2','EXT_SOURCE_3'],axis=1)

In [ ]:
app_score_col_remove.shape

# Feature Engineering 

In [ ]:
#missing imputation

In [ ]:
app_score_col_remove.isnull().sum().sort_values()/app_score_col_remove.shape[0]

In [ ]:
app_score_col_remove.groupby('CNT_FAM_MEMBERS').size()

In [ ]:
app_score_col_remove['CNT_FAM_MEMBERS'].mode() #categorial columns

In [ ]:
app_score_col_remove['CNT_FAM_MEMBERS'] = app_score_col_remove['CNT_FAM_MEMBERS'].fillna((app_score_col_remove['CNT_FAM_MEMBERS'].mode()[0])) 

In [ ]:
app_score_col_remove['CNT_FAM_MEMBERS'].isnull().sum()

In [ ]:
app_score_col_remove.groupby('OCCUPATION_TYPE').size().sort_values()# 2 options fill null values with NA or labouborers

In [ ]:
app_score_col_remove['OCCUPATION_TYPE'] = app_score_col_remove['OCCUPATION_TYPE'].fillna((app_score_col_remove['OCCUPATION_TYPE'].mode()[0])) 

In [ ]:
app_score_col_remove['OCCUPATION_TYPE'].isnull().sum()

In [ ]:
app_score_col_remove['NAME_TYPE_SUITE'].isnull().sum()

In [ ]:
app_score_col_remove['NAME_TYPE_SUITE'] = app_score_col_remove['NAME_TYPE_SUITE'].fillna((app_score_col_remove['NAME_TYPE_SUITE'].mode()[0])) 

In [ ]:
app_score_col_remove['NAME_TYPE_SUITE'].isnull().sum()

In [ ]:
app_score_col_remove['AMT_ANNUITY'].describe()# continous cols hence impute with mean

In [ ]:
app_score_col_remove['AMT_ANNUITY'] = app_score_col_remove['AMT_ANNUITY'].fillna((app_score_col_remove['AMT_ANNUITY'].mean())) 

In [ ]:
app_score_col_remove['AMT_ANNUITY'].isnull().sum()

In [ ]:
app_score_col_remove['AMT_REQ_CREDIT_BUREAU_HOUR'].dtype() # float means continous columns

In [ ]:
app_score_col_remove['AMT_REQ_CREDIT_BUREAU_HOUR'].describe()

In [ ]:
amt_req_col =[]

for col in app_score_col_remove.columns:
    if col.startswith("AMT_REQ_CREDIT_BUREAU"):
          amt_req_col.append(col)  

amt_req_col            

In [ ]:
for col in amt_req_col:
    app_score_col_remove[col] = app_score_col_remove[col].fillna((app_score_col_remove[col].median())) 

In [ ]:
app_score_col_remove[col].isnull().sum()

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].isnull().sum()

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].describe()

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].agg(['min','max','median'])

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].mean()

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'] = app_score_col_remove['AMT_GOODS_PRICE'].fillna((app_score_col_remove['AMT_GOODS_PRICE'].median()))

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].isnull().sum()

# Value Modification

In [ ]:
days_col=[]

for col in app_score_col_remove.columns:
    if col.startswith("DAYS"):
        days_col.append(col)
days_col                                   #Negative value rectification

In [ ]:
for col in days_col:
    app_score_col_remove[col] = abs(app_score_col_remove[col])

In [ ]:
app_score_col_remove.head()

# Outlier detection & Treatement

In [ ]:
 app_score_col_remove['AMT_GOODS_PRICE'].agg(['min','max','median']) #Make binning

In [ ]:
sns.kdeplot(data=app_score_col_remove,x='AMT_GOODS_PRICE')

In [ ]:
sns.boxenplot(data=app_score_col_remove,x='AMT_GOODS_PRICE')

In [ ]:
app_score_col_remove['AMT_GOODS_PRICE'].quantile([0.1,.2,.3,.4,.5,.6,.7,.8,.9,.99]) # most ofthe values are below 1.8 million

In [ ]:
bins = [0,100000,200000,300000,400000,500000,600000,700000,800000,900000,4050000]
ranges = ['0-100K','100k-200K','200K-300K','300K-400K','400K-500K','500K-600K','600K-700K'
          ,'700K-800K','800K-900K','Above 900K']

app_score_col_remove['AMT_GOODS_PRICE_RANGE'] = pd.cut(app_score_col_remove['AMT_GOODS_PRICE'],bins,labels=ranges)

In [ ]:
app_score_col_remove.groupby(['AMT_GOODS_PRICE_RANGE']).size()

In [ ]:
app_score_col_remove['AMT_INCOME_TOTAL'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99])

In [ ]:
app_score_col_remove['AMT_INCOME_TOTAL'].max()

In [ ]:
bins = [0,100000,150000,200000,250000,300000,350000,400000,117000000]
ranges = ['0-100K','100K-150K','150K-200K','200K-250K','250K-300K','300K-350K','350K-400K'
          ,'Above 400K']

app_score_col_remove['AMT_INCOME_TOTAL_RANGE'] = pd.cut(app_score_col_remove['AMT_INCOME_TOTAL'],bins,labels=ranges)


In [ ]:
app_score_col_remove.groupby(['AMT_INCOME_TOTAL_RANGE']).size()


In [ ]:

app_score_col_remove['AMT_INCOME_TOTAL_RANGE'].isnull().sum()

In [ ]:
app_score_col_remove['AMT_CREDIT'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99])


In [ ]:

app_score_col_remove['AMT_CREDIT'].max()

In [ ]:
# In[55]:


bins = [0,200000,400000,600000,800000,900000,1000000,2000000,3000000,4050000]
ranges = ['0-200K','200K-400K','400K-600K','600K-800K','800K-900K','900K-1M','1M-2M','2M-3M','Above 3M']

app_score_col_remove['AMT_CREDIT_RANGE'] = pd.cut(app_score_col_remove['AMT_CREDIT'],bins,labels=ranges)

In [ ]:
app_score_col_remove.groupby(['AMT_CREDIT_RANGE']).size()

In [ ]:
app_score_col_remove['AMT_CREDIT'].isnull().sum()


In [ ]:
app_score_col_remove['AMT_ANNUITY'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99])

In [ ]:
# In[59]:
app_score_col_remove['AMT_ANNUITY'].max()

In [ ]:
# In[60]:
bins = [0,25000,50000,100000,150000,200000,258025.5]
ranges = ['0-25K','25K-50K','50K-100K','100K-150K','150K-200K','Above 200K']

app_score_col_remove['AMT_ANNUITY_RANGE'] = pd.cut(app_score_col_remove['AMT_ANNUITY'],bins,labels=ranges)



In [ ]:
app_score_col_remove.groupby(['AMT_ANNUITY_RANGE']).size()

In [ ]:
# In[62]:remove
app_score_col_remove['AMT_ANNUITY_RANGE'].isnull().sum()

In [ ]:
app_score_col_remove['DAYS_EMPLOYED'].agg(['min','max','median'])

In [ ]:
app_score_col_remove['DAYS_EMPLOYED'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.81,0.85,0.9,0.95,0.99])

In [ ]:

app_score_col_remove[app_score_col_remove['DAYS_EMPLOYED']<app_score_col_remove['DAYS_EMPLOYED'].max()].max()['DAYS_EMPLOYED']


In [ ]:
app_score_col_remove['DAYS_EMPLOYED'].max()

In [ ]:

bins = [0,1825,3650,5475,7300,9125,10950,12775,14600,16425,18250,23691,365243]

ranges = ['0-5Y','5Y-10Y','10Y-15Y','15Y-20Y','20Y-25Y','25Y-30Y','30Y-35Y','35Y-40Y','40Y-45Y','45Y-50Y'
          ,'50Y-65Y','Above 65Y']

app_score_col_remove['DAYS_EMPLOYED_RANGE'] = pd.cut(app_score_col_remove['DAYS_EMPLOYED'],bins,labels=ranges)

In [ ]:

app_score_col_remove.groupby(['DAYS_EMPLOYED_RANGE']).size()

In [ ]:

app_score_col_remove['DAYS_BIRTH'].quantile([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.81,0.85,0.9,0.95,0.99])

In [ ]:
app_score_col_remove['DAYS_BIRTH'].min()

In [ ]:
bins = [0,7300,10950,14600,18250,21900,25229]

ranges = ['20Y','20Y-30Y','30Y-40Y','40Y-50Y','50Y-60Y','Above 60Y']

app_score_col_remove['DAYS_BIRTH_RANGE'] = pd.cut(app_score_col_remove['DAYS_BIRTH'],bins,labels=ranges)

In [ ]:

app_score_col_remove.groupby(['DAYS_BIRTH_RANGE']).size()

In [ ]:
app_score_col_remove['DAYS_BIRTH'].isnull().sum()

# Data Analysis

In [ ]:
app_score_col_remove.info()

In [ ]:
app_score_col_remove.dtypes.value_counts()

In [ ]:
obj_var=app_score_col_remove.select_dtypes(include=['object']).columns
obj_var       #putting object columns in Df

In [ ]:
app_score_col_remove.groupby(['NAME_CONTRACT_TYPE']).size()

In [ ]:
sns.countplot(data=app_score_col_remove,x='NAME_CONTRACT_TYPE',hue='TARGET')

In [ ]:
data_pct=app_score_col_remove[['NAME_CONTRACT_TYPE','TARGET']].groupby(['NAME_CONTRACT_TYPE'],as_index=False).mean()
data_pct

In [ ]:
data_pct['pct']=data_pct['TARGET']*100 #creating a new column for %

In [ ]:
data_pct

In [ ]:
sns.barplot(data=data_pct,x='NAME_CONTRACT_TYPE',y='pct') #percentage of cash loan defaulters are more 

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)

sns.barplot(data=data_pct,x='NAME_CONTRACT_TYPE',y='pct') 

plt.subplot(1,2,2)
sns.countplot(data=app_score_col_remove,x='NAME_CONTRACT_TYPE',hue='TARGET')

In [ ]:
plt.figure(figsize=(25,60)) 

for i,var in enumerate (obj_var):#var=NAmecontracttype, i=target
    data_pct=app_score_col_remove[[var,'TARGET']].groupby([var],as_index=False).mean().sort_values(by='TARGET',ascending=False)
    data_pct['pct']=data_pct['TARGET']*100
    
    plt.subplot(10,2,i+i+1)
    plt.subplots_adjust(wspace=0.1,hspace=1)  #gives spacing between graphs   
    sns.countplot(data=app_score_col_remove,x=var,hue='TARGET')
    plt.xticks(rotation=90) 
    
    plt.subplot(10,2,i+i+2)
    sns.barplot(data=data_pct,x=var,y='pct') 
    plt.xticks(rotation=90)   #line variable in 90 degree

  

In [ ]:
#Now moving to categorical variables

In [ ]:
app_score_col_remove.dtypes.value_counts()

In [ ]:
num_var = app_score_col_remove.select_dtypes(include=['float64','int64']).columns
num_cat_var = app_score_col_remove.select_dtypes(include=['float64','int64','category']).columns
num_var

In [ ]:
len(num_var)

In [ ]:
num_data=app_score_col_remove[num_var]
num_data.groupby(['TARGET']).size()/num_data.shape[0]*100

In [ ]:
# Data Imbalance so making 2 diff datasets

In [ ]:
defaulters=num_data[num_data['TARGET']==1]

In [ ]:
defaulters

In [ ]:
repayers=num_data[num_data['TARGET']==0]

In [ ]:
repayers

In [ ]:
#dropping target column 

In [ ]:
defaulters=num_data[num_data['TARGET']==1].drop(['TARGET'],axis=1)
repayers=num_data[num_data['TARGET']==0].drop(['TARGET'],axis=1)

In [ ]:
defaulter_corr =defaulters.corr()

In [ ]:
defaulter_corr.head()

In [ ]:
defaulter_corr_unstck = defaulter_corr.where(np.triu(np.ones(defaulter_corr.shape),k=1).astype(np.bool)).unstack().reset_index().rename(columns={'level_0':'var1' ,'level_1':'var2',0:'corr'})

In [ ]:
defaulter_corr_unstck.head()

In [ ]:
defaulter_corr_unstck['corr'] = abs(defaulter_corr_unstck['corr'])
defaulter_corr_unstck.dropna(subset=['corr']).sort_values(by=['corr'],ascending=False).head(10)

In [ ]:
repayers_corr = repayers.corr()
repayers_corr_unstck = repayers_corr.where(np.triu(np.ones(repayers_corr.shape),k=1).astype(np.bool)).unstack().reset_index().rename(columns={'level_0':'var1'
                                                                                                                        ,'level_1':'var2'
                                                                                                                        ,0:'corr'})
repayers_corr_unstck['corr'] = abs(repayers_corr_unstck['corr'])
repayers_corr_unstck.dropna(subset=['corr']).sort_values(by=['corr'],ascending=False).head(10)


In [ ]:
amt_var = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE']


In [ ]:
sns.kdeplot(data=num_data,x='AMT_CREDIT',hue='TARGET')

In [ ]:
plt.figure(figsize=(10,5))

for i, col in enumerate(amt_var):
    plt.subplot(2,2,i+1)
    sns.kdeplot(data=num_data,x=col,hue='TARGET')
    plt.subplots_adjust(wspace=0.5,hspace=0.5)

In [ ]:
num_data

In [ ]:
sns.scatterplot(data=num_data,x='AMT_CREDIT',y='CNT_CHILDREN',hue='TARGET')


In [ ]:
amt_var = num_data[['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','TARGET']]

In [ ]:
sns.pairplot(data=amt_var,hue='TARGET')

# Previous Data

In [ ]:
null_count = pd.DataFrame(prev_app.isnull().sum().sort_values(ascending=False)/prev_app.shape[0]*100).reset_index().rename(columns={'index':'var',
                                                                                                                                   0:'count_pct'})

In [ ]:
null_count

In [ ]:
var_msng_ge_40 = list(null_count[null_count['count_pct']>=40]['var'])
var_msng_ge_40

In [ ]:
nva_cols = var_msng_ge_40+['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT','NFLAG_LAST_APPL_IN_DAY']
len(nva_cols)

In [ ]:
len(prev_app.columns)

In [ ]:
prev_app_nva_col_rmvd = prev_app.drop(labels=nva_cols,axis=1)

In [ ]:
len(prev_app_nva_col_rmvd.columns)

In [ ]:
prev_app_nva_col_rmvd.columns

In [ ]:

prev_app_nva_col_rmvd.head()

In [ ]:

prev_app_nva_col_rmvd.isnull().sum().sort_values(ascending=False)/prev_app_nva_col_rmvd.shape[0]*100

In [ ]:

prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].agg(func=['mean','median'])


In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].mode()

In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE_MEDIAN'] = prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].fillna(prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].median())

In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE_MEAN'] = prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].fillna(prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].mean())


In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE_MODE'] = prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].fillna(prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].mode()[0])

In [ ]:
gp_cols = ['AMT_GOODS_PRICE','AMT_GOODS_PRICE_MEDIAN','AMT_GOODS_PRICE_MEAN','AMT_GOODS_PRICE_MODE']

In [ ]:
plt.figure(figsize=(10,5))

for i, col in enumerate(gp_cols):
    plt.subplot(2,2,i+1)
    sns.kdeplot(data=prev_app_nva_col_rmvd,x=col)
    plt.subplots_adjust(wspace=0.5,hspace=0.5)  #choose the method which is close to the original values i,e Median

In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE'] = prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].fillna(prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].median())



In [ ]:
prev_app_nva_col_rmvd['AMT_GOODS_PRICE'].isnull().sum()


In [ ]:
prev_app_nva_col_rmvd['AMT_ANNUITY'].agg(func=['mean','median','max'])

In [ ]:
prev_app_nva_col_rmvd['AMT_ANNUITY'] = prev_app_nva_col_rmvd['AMT_ANNUITY'].fillna(prev_app_nva_col_rmvd['AMT_ANNUITY'].median())

In [ ]:
prev_app_nva_col_rmvd['PRODUCT_COMBINATION'] = prev_app_nva_col_rmvd['PRODUCT_COMBINATION'].fillna(prev_app_nva_col_rmvd['PRODUCT_COMBINATION'].mode()[0])


In [ ]:
prev_app_nva_col_rmvd['CNT_PAYMENT'].agg(func=['mean','median','max'])


In [ ]:
prev_app_nva_col_rmvd[prev_app_nva_col_rmvd['CNT_PAYMENT'].isnull()].groupby(['NAME_CONTRACT_STATUS']).size().sort_values(ascending=False)




In [ ]:

prev_app_nva_col_rmvd['CNT_PAYMENT'] = prev_app_nva_col_rmvd['CNT_PAYMENT'].fillna(0)


In [ ]:
prev_app_nva_col_rmvd.isnull().sum().sort_values(ascending=False)


In [ ]:
prev_app_nva_col_rmvd = prev_app_nva_col_rmvd.drop(labels=['AMT_GOODS_PRICE_MEDIAN','AMT_GOODS_PRICE_MEAN','AMT_GOODS_PRICE_MODE'],axis=1)


In [ ]:
prev_app_nva_col_rmvd.isnull().sum().sort_values(ascending=False)


In [ ]:
len(prev_app_nva_col_rmvd.columns)


In [ ]:
prev_app_nva_col_rmvd.head()

In [ ]:


merged_df = pd.merge(app_score_col_remove,prev_app_nva_col_rmvd,how='inner',on='SK_ID_CURR')
merged_df.head()


In [ ]:
plt.figure(figsize=(15,5))

sns.countplot(data=merged_df,x='NAME_CASH_LOAN_PURPOSE',hue='NAME_CONTRACT_STATUS')
plt.xticks(rotation=90)
plt.yscale('log')


In [ ]:
sns.countplot(data=merged_df,x='NAME_CONTRACT_STATUS',hue='TARGET')

In [ ]:
merged_agg = merged_df.groupby(['NAME_CONTRACT_STATUS','TARGET']).size().reset_index().rename(columns={0:'counts'})
sum_df  = merged_agg.groupby(['NAME_CONTRACT_STATUS'])['counts'].sum().reset_index()

merged_agg_2 = pd.merge(merged_agg,sum_df,how='left',on='NAME_CONTRACT_STATUS')
merged_agg_2['pct'] = round(merged_agg_2['counts_x']/merged_agg_2['counts_y']*100,2)
merged_agg_2


In [ ]:
sns.lineplot(data=merged_df,x='NAME_CONTRACT_STATUS',y='AMT_INCOME_TOTAL',ci=None,hue='TARGET')



In [ ]:

len(merged_df.columns)

In [ ]:
# # All the analysis

# most of the customers have taken cash loan
# customers who have taken cash loans are less likely to default
# 
# CODE_GENDER - 
# 
#     most of the loans have been taken by female
#     default rate for females are just ~7% which is safer and lesser than male
# 
# NAME_TYPE_SUITE - 
# 
#     unacompanied people had tanke most of the loans and the default rate is ~8.5% which is still okay
# 
# NAME_INCOME_TYPE - 
# 
#     the safest segments are working, commercial associates and pensioners
# 
# NAME_EDUCATION_TYPE - 
# 
#     Higher education is the safest segment to give the loan with a default rate of less than 5%
# 
# NAME_FAMILY_STATUS - 
# 
#     Married people are safe to target, default rate is 8%
# 
# 
# NAME_HOUSING_TYPE - 
# 
#     People having house/appartment are safe to give the loan with default rate of ~8%
# 
# OCCUPATION_TYPE - 
# 
#     Low-Skill Laboreres and drivers are highest defaulters
#     Accountants are less defaulters
#     Core staff, Managers and Laborers are safer to target with a default rate of <= 7.5 to 10%
# 
# ORGANIZATION_TYPE - 
# 
#     Transport type 3 highest defaulter
#     Others, Business Entity Type 3, Self Employed are good to go with default rate around 10 %
# 
# =======univariate numeric variables analysis========
# 
#     >> most of the loans were given for the goods price ranging between 0 to 1 ml
#     >> most of the loans were given for the credit amount of 0 to 1 ml
#     >> most of the customers are paying annuity of 0 to 50 K
#     >> mostly the customers have income between 0 to 1 ml
# 
# =============bivariate analysis==================
# 
#     >> AMT_CREDIT and AMT_GOODS_PRICE are linearly corelated, if the AMT_CREDIT increases the defaulters are decreasing
#     >> people having income less than or equals to 1 ml, are more like to take loans out of which who are taking loan of less than 1.5 million, coudl turn out to be defaulters. we can target income below 1 million and loan maount greater than 1.5 million
#     >> people having children 1 to less than 5 are safer to give the loan
#     >> People who can pay the annuity of 100K are more like to get the loan and that's upto less than 2ml (safer segment)
# 
# ============analysis on merged data==============
# 
#     >> for the repairing purpose customers had applied mostly prev. and the same puspose has most number of cancelations
#     >> most of the app. which were prev. either canceled or refused 80-90% of them are repayer in the current data
#     >> offers which were unused prev. now have maximum number of defaulters despite of having high income band customers

